In [ ]:
# instalação dos pacotes necessários na versão mais nova
%pip install -U scikit-learn pandas numpy seaborn

In [2]:
# importando bibliotecas necessárias
import pandas as pd
import numpy as np

# graphs
import matplotlib.pyplot as plt
import seaborn as sns

# separação de dados
from sklearn.model_selection import train_test_split

# padronização dos dados
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# construção de filtro para ColumnTransformer
from sklearn.compose import make_column_selector

# criação de pipeline para tratamento do conjunto de dados
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# modelo
from sklearn.linear_model import LassoCV, RidgeCV

In [3]:
# agora uma função que vai printar bonitinho as métricas
def print_metrics(model: Pipeline, model_name: str, X_test: np.ndarray, y_test: np.ndarray, y_pred: np.ndarray) -> str:
  """
  Returns the accuracy, precision and recall scores of the model
      
      Arguments:
          model_name {str} -- name of the model
          y_test {array} -- array of the test labels
          y_pred {array} -- array of the predicted labels
          y_test_pca {array} -- array of the test labels after PCA
          y_pred_pca {array} -- array of the predicted labels after PCA
      
      Returns:
          A formatted string containing the accuracy, precision and recall scores of the model for both non-PCA and PCA.
  
  """
  #imports
  from sklearn.metrics import mean_absolute_error, mean_squared_error

  return (f'''

Mean Absolute Error (MAE) do Modelo {model_name} sem PCA: {mean_absolute_error(y_test, y_pred):.3f}

Mean Squared Error (MSE) do Modelo {model_name} sem PCA: {mean_squared_error(y_test, y_pred, squared=True):.4f}

Root Mean Squared Error (RMSE) do Modelo {model_name} sem PCA: {mean_squared_error(y_test, y_pred, squared=False):.4f}

R-squared do Modelo {model_name} sem PCA: {model.score(X_test, y_test):.3f}
''')

In [4]:
def get_features(model: Pipeline, model_name: str, ispca: bool) -> str:
  """
  # Get Features Function
  
  Returns a string containing the number of original features, the number of features selected by the model (in case the model is a Lasso) and the name of the features selected (will return all features if it's not a Lasso).
      
      ### Arguments:

          - model {Pipeline} -- a pipeline object containing an estimator/model with fit/transform/predict operations.

          - model_name {str} -- name of the model used inside the pipeline to identify the model

          - ispca {bool} -- if the model passed is a model with pca, this should be true, otherwise it should be false.
      
      ### Returns:
      
          Returns a string containing the number of original features, the number of features selected by the model (in case the model is a Lasso) and the name of the features selected (will return all features if it's not a Lasso).
  
  """

  preprocessor_fitted = model['preprocessor']
  
  if ispca:
    feature_names = (
      list(preprocessor_fitted.named_transformers_['num']['pca'].get_feature_names_out()) +
      list(preprocessor_fitted.named_transformers_['cat']['pca'].get_feature_names_out())
      )
  
  else:
    feature_names = (
      list(preprocessor_fitted.named_transformers_['num']['imputer'].get_feature_names_out()) +
      list(preprocessor_fitted.named_transformers_['cat']['one_hot_encoder'].get_feature_names_out())
      )
  feature_names = np.array(feature_names)
  kept = ~np.isclose(model[model_name].coef_, 0)
  
  return(f'''
    Número de Features Original: {len(feature_names)}
    Número de Features selecionadas pelo {model_name}: {len(feature_names[kept])}
    Features Selecionadas:
    {feature_names[kept]}
    ''')

In [5]:
from warnings import simplefilter
#from sklearn.exceptions import UserWarning

simplefilter("ignore", category=UserWarning)

## 1. Análise exploratória de dados (1.0 ponto)

Realize a análise exploratória de dados. Observe cada variável, como se comportam em relação a outras variáveis independentes e com a variável dependente.


In [ ]:
# carregando dataset de teste

dataframe = pd.read_csv('dado/train.csv')
dataframe_test = pd.read_csv('dado/test.csv')
dataframe.info()

In [7]:
# Separando dados de treinamento do modelo
X = dataframe.drop(['SalePrice'], axis=1)
y = dataframe.SalePrice

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 2. Levantamento de hipóteses (1.0 ponto)

Descreva quais hipóteses você observou ao fazer a análise exploratória de dados.



## 3. Apresentação das ideias obtidas (1.0 ponto)

Apresente com gráficos as suas observações e a descreva cada gráfico.


## 4. Preparação dos dados (1.0 ponto)

Faça um processamento nos dados, preenchendo valores faltantes, removendo dados ou variáveis inconsistentes e normalizado os dados



#### Passos
- Outliers
- Encoding -> `OneHotEncoder`
- Impute Missing Values -> `SimpleImputer`
- Scaler -> `StandardScaler`
- PCA -> `PCA`
- Model -> `LassoCV`
- Cross-Val -> _Interno no_ `LassoCV`

O `OneHotEncoder` é utilizado para codificar as variáveis categóricas em variáveis numéricas (mas ainda mantendo a propriedade de categoria, digamos assim). O seu funcionamento é parecido com o método `get_dummies` do Pandas, porém mais simples de ser aplicado e é possível utilizá-lo dentro de um `ColumnTransformer` que, por sua vez, é usado num `Pipeline` para automatizar as tarefas de pré-processamento e treinamento do modelo. Outra opção seria utilizar o `OrdinalEncoder` porém a própria documentação aponta que o mesmo pode afetar o desempenho de modelos lineares, tais como o LassoCV e o RidgeCV.

In [8]:
# filtros para o pipeline

col_num = make_column_selector(dtype_include=np.number)
col_tex = make_column_selector(dtype_include=object)

# construindo pipeline para o modelo de regressão usando LassoCV sem pca

numeric_transformer = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="median")),
          ("scaler", StandardScaler()),
        ]
)

categorical_transformer = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
          ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False)),
        ]
)

preprocessor = ColumnTransformer(
  transformers=[
    ("num", numeric_transformer, col_num),
    ("cat", categorical_transformer, col_tex)
  ]
)

modelLassoCV = Pipeline(
  steps=[
    ("preprocessor", preprocessor), 
    ("lassocv", LassoCV(cv=5))
  ]
)

modelLassoCV

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb66493f6a0>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb66493f700>)])),
                ('lassocv', LassoCV(cv=5))])

In [9]:
# fit e predict do modelo
modelLassoCV.fit(X_train, y_train)
y_pred = modelLassoCV.predict(X_test)

In [13]:
print(get_features(model=modelLassoCV, model_name='lassocv', ispca= False))


    Número de Features Original: 300
    Número de Features selecionadas pelo lassocv: 90
    Features Selecionadas:
    ['Id' 'MSSubClass' 'LotFrontage' 'LotArea' 'OverallQual' 'OverallCond'
 'YearBuilt' 'YearRemodAdd' 'MasVnrArea' 'BsmtFinSF1' 'BsmtFinSF2'
 'TotalBsmtSF' '2ndFlrSF' 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath'
 'BsmtHalfBath' 'FullBath' 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr'
 'TotRmsAbvGrd' 'Fireplaces' 'GarageYrBlt' 'GarageCars' 'GarageArea'
 'WoodDeckSF' 'OpenPorchSF' '3SsnPorch' 'ScreenPorch' 'PoolArea' 'MiscVal'
 'MoSold' 'x0_RM' 'x3_IR2' 'x4_Bnk' 'x4_HLS' 'x6_CulDSac' 'x7_Mod'
 'x8_BrkSide' 'x8_Crawfor' 'x8_Edwards' 'x8_Gilbert' 'x8_Mitchel'
 'x8_NAmes' 'x8_NWAmes' 'x8_NoRidge' 'x8_NridgHt' 'x8_Somerst'
 'x8_StoneBr' 'x9_Norm' 'x10_PosN' 'x11_1Fam' 'x11_2fmCon' 'x12_1Story'
 'x12_2Story' 'x13_Gable' 'x14_ClyTile' 'x15_BrkFace' 'x15_HdBoard'
 'x16_ImStucc' 'x16_Plywood' 'x17_BrkFace' 'x17_None' 'x18_Ex' 'x18_TA'
 'x20_BrkTil' 'x20_PConc' 'x21_Ex' 'x22_TA' 'x23_Gd'

In [199]:
# pontuações
print(f'Pontuação do Modelo: {print_metrics(model=modelLassoCV, model_name="Pipeline LassoCV", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline LassoCV sem PCA: 17523.065

Mean Squared Error (MSE) do Modelo Pipeline LassoCV sem PCA: 866285046.5730

Root Mean Squared Error (RMSE) do Modelo Pipeline LassoCV sem PCA: 29432.7207

R-squared do Modelo Pipeline LassoCV sem PCA: 0.887



## 5. Criação e seleção de características (1.0 ponto)

Observando as variáveis disponíveis, é possível criar novas variáveis? Ou trazer de um dado externo novos dados para agregar aos existentes? É possível selecionar uma variável ou grupo de variáveis que melhor explica a variável dependente?


Aqui eu faço pipeline com pca

In [14]:
# filtros para o pipeline com pca

col_num = make_column_selector(dtype_include=np.number)
col_tex = make_column_selector(dtype_include=object)

# construindo pipeline para o modelo de regressão usando LassoCV

numeric_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="median")),
          ("scaler", StandardScaler()),
          ("pca", PCA(n_components='mle'))
        ]
)

categorical_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
          ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False)),
          ("pca", PCA(n_components='mle'))
        ]
)

preprocessor = ColumnTransformer(
  transformers=[
    ("num", numeric_transformer_pca, col_num),
    ("cat", categorical_transformer_pca, col_tex)
  ]
)

modelLassoCV_pca = Pipeline(
  steps=[
    ("preprocessor", preprocessor), 
    ("lassocv", LassoCV(cv=5))
  ]
)

modelLassoCV_pca

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb66493fc40>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb66493e6b0>)])),
                ('lassocv', LassoCV(cv=5))])

In [15]:
# fit e predict do modelo LassoCV com pca
modelLassoCV_pca.fit(X_train, y_train)
y_pred = modelLassoCV_pca.predict(X_test)

In [17]:
print(get_features(model=modelLassoCV_pca, model_name='lassocv', ispca= True))


    Número de Features Original: 250
    Número de Features selecionadas pelo lassocv: 108
    Features Selecionadas:
    ['pca0' 'pca1' 'pca2' 'pca3' 'pca4' 'pca5' 'pca6' 'pca7' 'pca8' 'pca9'
 'pca10' 'pca11' 'pca14' 'pca15' 'pca16' 'pca17' 'pca18' 'pca19' 'pca20'
 'pca22' 'pca23' 'pca24' 'pca25' 'pca26' 'pca27' 'pca28' 'pca29' 'pca31'
 'pca32' 'pca33' 'pca34' 'pca0' 'pca1' 'pca2' 'pca3' 'pca4' 'pca5' 'pca6'
 'pca7' 'pca8' 'pca9' 'pca10' 'pca12' 'pca13' 'pca14' 'pca15' 'pca16'
 'pca17' 'pca18' 'pca19' 'pca20' 'pca22' 'pca23' 'pca24' 'pca25' 'pca26'
 'pca27' 'pca29' 'pca32' 'pca33' 'pca34' 'pca35' 'pca36' 'pca39' 'pca40'
 'pca41' 'pca42' 'pca43' 'pca44' 'pca45' 'pca47' 'pca48' 'pca51' 'pca52'
 'pca53' 'pca55' 'pca56' 'pca57' 'pca59' 'pca60' 'pca61' 'pca62' 'pca64'
 'pca67' 'pca68' 'pca69' 'pca71' 'pca75' 'pca76' 'pca78' 'pca82' 'pca85'
 'pca86' 'pca88' 'pca89' 'pca94' 'pca96' 'pca98' 'pca99' 'pca102' 'pca104'
 'pca106' 'pca107' 'pca118' 'pca131' 'pca180' 'pca182' 'pca184']
    


In [207]:
# pontuações
print(f'Pontuação do Modelo: {print_metrics(model=modelLassoCV_pca, model_name="Pipeline LassoCV", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline LassoCV sem PCA: 18956.904

Mean Squared Error (MSE) do Modelo Pipeline LassoCV sem PCA: 978705610.2881

Root Mean Squared Error (RMSE) do Modelo Pipeline LassoCV sem PCA: 31284.2710

R-squared do Modelo Pipeline LassoCV sem PCA: 0.872



### Testando o Pipeline com Ridge e pca

In [19]:
# filtros para o pipeline com ridge e pca

# col_num = make_column_selector(dtype_include=np.number)
# col_tex = make_column_selector(dtype_include=object)

# construindo pipeline para o modelo de regressão usando LassoCV

# numeric_transformer_pca = Pipeline(
#   steps=[
#           ("imputer", SimpleImputer(strategy="median")),
#           ("scaler", StandardScaler()),
#           ("pca", PCA(n_components='mle'))
#         ]
# )

# categorical_transformer_pca = Pipeline(
#   steps=[
#           ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
#           ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False)),
#           ("pca", PCA(n_components='mle'))
#         ]
# )

# preprocessor = ColumnTransformer(
#   transformers=[
#     ("num", numeric_transformer_pca, col_num),
#     ("cat", categorical_transformer_pca, col_tex)
#   ]
# )

modelRidgeCV_pca = Pipeline(
  steps=[
    ("preprocessor", preprocessor), 
    ("lassocv", RidgeCV(cv=5))
  ]
)

modelRidgeCV_pca

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb66493fc40>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb66493e6b0>)])),
                ('lassocv', RidgeCV(cv=5))])

## 6. Decisão do modelo (1.0 ponto)

Dada toda a análise realizada nos pontos anteriores, qual modelo melhor se adequa a este problema? E por quê?



aqui eu escolho o pipeline com melhor resultado

## 7. Modelagem (1.0 ponto)

Divida os dados em teste e treino, construa o seu modelo e teste alterar os parâmetros utilizados por ele.



aqui eu fito e printo as métricas de novo
se pá uso um  GridSearchCV

## 8. Histórico (1.0 pontos)

Preencha a tabela abaixo com cada teste realizado com seu modelo. Não se preocupe caso voce mude de ideia sobre qual modelo testar, mas não deixe de testá-lo exaustivamente e, caso precise, retorne para o passo de criação e seleção de características.


tabelinha de métricas
#testar ridge

## 9. Resultado (2.0 pontos)

Reúna os seus achados, os valores das métricas do seu melhor modelo obtido e o melhor conjunto de variáveis. Escreva abaixo um relatório que seria enviado para uma diretoria de uma empresa sobre seu trabalho.


escolher